<a href="https://colab.research.google.com/github/davetew/Modern-Aerospace-Propulsion/blob/main/Week%2005%20-%20Combustors%20%26%20Augmentors/Homework_2_Combustors_%26_Augementors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework \#2 - Combustors and Augmentors
Please evaluate the turbojet performance implications in terms of the specific impulse ($\frac{T}{\dot{m}_fg}$) and overall efficiency for four candidate climate-friendly fuels ($H_2$, $NH_3$, $CH_4$, $CH_3OH$) and $Jet-A$.  

However, in doing so, please use the appropriate combustor exhaust gas properties (i.e., $\gamma$, $R$) in your evaluation of the performance of the turbine and nozzle.

| Parameter | Value |
|---|---|
| $M_\infty$ | 0.85 |
| Alitude (km) | 10 |
| Compressor Pressure Ratio ($\pi_c$) | 25 |
| Compressor Isentropic Efficiency ($\eta_c$) | 0.85 |
| Turbine Isentropic Efficiency ($\eta_t$) | 0.85 |
| Turbine Inlet Temperature (K) | 1500 |

Please use $M_\infty$, the ambient static temperature at 10 km, $\pi_c$, and $\eta_c$ to calculate the combustor inlet total conditions.  You may then use the `Combustion` class defined below to determine the fuel/air mass ratio for each fuel that is required to yield the design turbine inlet temperature.

Partially expand the combustion products through the turbine to drive the compressor (i.e., $w_t = -w_c$).  However, in doing so, please use the combustor exit gas properties (i.e., $\gamma$, $R$), assuming that they are constant through the turbine and nozzle (i.e., the "frozen flow" assumption).  Then expand the turbine exit flow through a nozzle to ambient static pressure.

For each fuel, calculate the overall efficiency and specific impulse of the turbojet.

I have tried to help you set up the problem below by providing some helpful python code -- in an effort to keep the focus on the propulsion and not the coding.  However, please feel free to build and use your own tools.

In [13]:
# Quietly install cantera (Combustion) and pint (Units) -- both non-standard Colab packages
%pip install -q cantera pint ambiance

In [18]:
# Import the required python packages
import numpy as np
import cantera as ct
from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina' # Mac retina display resolution for plots

from typing import Dict, Literal, Tuple, Union, List

import pandas as pd
import pint

# Save default unit registry in ureg & Quantity in Q_
ureg = pint.UnitRegistry()
Q_ = ureg.Quantity

from ambiance import Atmosphere

In [6]:
class Combustion:
  """Class to facilitate the equilibrium analysis of combustion reactions with Cantera"""

  # Extract all species in the NASA database
  full_species = ct.Species.list_from_file('nasa_gas.yaml')

  # Select all species with C, O, N, or H atoms
  species = []
  for spec in full_species:
      elements = spec.composition.keys()
      if all(elem in ['C', 'O', 'N', 'H'] for elem in elements):
          species.append(spec)

  def __init__(self, fuel: Dict[str, float] = {'Jet-A(g)': 1.0},
               oxidizer: Dict[str, float] = {'O2': 1.0, 'N2': 3.76},
               composition_basis: Literal['mass', 'mole'] = 'mole',
               phi: float = 1.0, mechanism: Union[str, None] = None) -> None:
    """Initialize an instance of Combustion given
      - fuel (Dict): The fuel composition,
      - oxidizer (Dict): The oxidizer composition,
      - composition_basis (Literal['mass' or 'mole']: The basis by which the fuel and oxidizer compositions are specified,
      - phi (float): The ratio of the fuel to air mass ratio to the stoichiometric fuel to air mass ratio,
      - mechanims Union[str, None]: The cantera mechanism to use for properties with the default being the NASA CEA gas properties"""

    # Save the input parameters
    self.fuel = fuel
    self.oxidizer = oxidizer
    self.composition_basis = composition_basis
    self.phi = phi
    self.mechanism = mechanism

    # Initialize an instance of a Cantera solution
    if mechanism is None:
      # Use the default NASA CEA gaseous species properties
      self.phase = ct.Solution(thermo='ideal-gas', species=self.species)
    else:
      # Use the specified mechanism
      self.phase = ct.Solution(mechanism)

  def Solution(self, name="Generic Solution"):
    """Return a Cantera solution using either the default NASA CEA properties or the specified mechanism"""
    if self.mechanism is None:
      # Use the default NASA CEA gaseous species properties
      return ct.Solution(thermo='ideal-gas', species=self.species, name=name)
    else:
      return ct.Solution(self.mechanism)

  def equilibrate(self, constant: Literal["HP", "TP", "TV"] = "HP",
                  phi: Union[None, float]= None, T_K: float = 298, p_bar: float = 1) -> ct.Solution:
    """Calculate and return the equilibrium composition at of the specified fuel given
    constant (Literal): the thermodynamic parameters to hold constant during the equilibration,
      - phi (float, None): the equivalence ratio,
      - T_K (float): the initial temperature in Kelvin,
      - p_bar (float): the initial pressure in bar"""

    # Use the default phi if not specified
    phi = self.phi if phi is None else phi

    reactants = self.Solution(name="Reactants")
    reactants.TP = T_K, p_bar * ct.one_atm
    reactants.set_equivalence_ratio(phi, self.fuel, self.oxidizer, self.composition_basis)

    # Equilibrate the mixture @ constant enthalpy and pressure
    reactants.equilibrate(constant)

    return reactants

  def R(self, constant: Literal["HP", "TP", "TV"] = "HP",
        phi: Union[None, float]= None,
        T_K: float = 298, p_bar: float = 1,
        basis: Literal['mole', 'mass'] = "mass") -> Q_:
    """Calculate and return the molar or mass basis gas constant for the combustion products"""

    # Equilibrate the reactants
    products = self.equilibrate(constant=constant, phi=phi, T_K=T_K, p_bar=p_bar)

    if basis == "mole":
      return Q_(products.cp_mole - products.cv_mole, "J/(mol*K)")
    elif basis == "mass":
      return Q_(products.cp_mass - products.cv_mass, "J/(kg*K)")
    else:
      raise ValueError("Basis must be either 'mole' or 'mass'")

  def ga(self, constant: Literal["HP", "TP", "TV"] = "HP",
        phi: Union[None, float]= None,
        T_K: float = 298, p_bar: float = 1) -> float:
    """Calculate and return the ratio of specific heats for the combustion products"""

    # Equilibrate the reactants
    products = self.equilibrate(constant=constant, phi=phi, T_K=T_K, p_bar=p_bar)

    return products.cp_mass / products.cv_mass

  @property
  def lower_heating_value(self) -> Q_:
    """Calculate and return the lower heating value of the specified fuel"""

    # Specify the reactant state
    reactants = self.Solution(name="Reactants")
    reactants.TP = 298, ct.one_atm
    reactants.set_equivalence_ratio(1.0, self.fuel, self.oxidizer, self.composition_basis)

    # Calculate the fuel mass fraction
    Y_fuel = reactants[self.fuel].Y[0]

    # Complete combustion product mole fractions
    X_products = {'CO2': reactants.elemental_mole_fraction('C'),
                  'H2O': 0.5 * reactants.elemental_mole_fraction('H'),
                  'N2': 0.5 * reactants.elemental_mole_fraction('N')}

    # Calculate the product enthalpy at 298 K, 1 atm
    products = self.Solution(name="Products")
    products.TPX = 298, ct.one_atm, X_products

    return Q_( (reactants.enthalpy_mass - products.enthalpy_mass) / Y_fuel,
              'J/kg').to_compact()

  def adiabatic_flame_temperature(self, phi: Union[None, float]= None, T_K: float = 298, p_bar: float = 1) -> Q_:
    """Calculate and return the adiabatic flame temperature of the specified fuel given
      - phi (float, None): the equivalence ratio,
      - T_K (float): the initial temperature in Kelvin,
      - p_bar (float): the initial pressure in bar"""

    # Equilibrate the reactants at the specified conditions
    products = self.equilibrate(constant="HP", phi=phi, T_K=T_K, p_bar=p_bar)

    # Return the temperature
    return Q_(products.T, "K")

  def products(self, phi: Union[None, float]= None, T_K: float = 298, p_bar: float = 1,
               composition_basis: Literal['mass', 'mole'] = 'mole') -> pd.Series:
    """Calculate and return the product composition as mole or mass fractions for
    a fuel/air reaction at constant enthalpy and pressure given
      - phi (float, None): the equivalence ratio,
      - T_K (float): the initial temperature in Kelvin,
      - p_bar (float): the initial pressure in bar
      - composition_basis (Literal['mass', 'mole']: The basis by which species fractions are to be returned."""

    # Equilibrate the reactants at the specified conditions
    products = self.equilibrate(constant="HP", phi=phi, T_K=T_K, p_bar=p_bar)

    # Return the product mole or mass fractions
    if composition_basis == "mole":
      return pd.Series({species: products[species].Y[0] for species in products.species_names})
    elif composition_basis == "mass":
      return pd.Series({species: products[species].X[0] * products.mean_molecular_weight for species in products.species_names})
    else:
      raise ValueError("Composition_basis must be either 'mass' or 'mole'")

    def __repr__(self):
      return f"Combustion(fuel={self.fuel}, \noxidizer={self.oxidizer}, \ncomposition_basis={self.composition_basis}, \nphi={self.phi}, \nmechanism={self.mechanism})"



In [17]:
fuels = ["Jet-A(g)", "H2", "NH3", "CH4", "CH3OH"]
data = pd.DataFrame(index=fuels, columns=["LHV (MJ/kg)", "Stoich Adiabatic Flame Temp (K)", "R (J/kg/K)", "gamma"])

for fuel in fuels:
  burner = Combustion(fuel={fuel: 1.0})
  data.loc[fuel, "LHV (MJ/kg)"] = burner.lower_heating_value.magnitude
  data.loc[fuel, "Stoich Adiabatic Flame Temp (K)"] = burner.adiabatic_flame_temperature().magnitude
  data.loc[fuel, "R (J/kg/K)"] = burner.R().magnitude
  data.loc[fuel, "gamma"] = burner.ga()

display(data)


,LHV (MJ/kg),Stoich Adiabatic Flame Temp (K),R (J/kg/K),gamma
Jet-A(g),43.351252,2279.305902,290.947993,1.251151
H2,119.95195,2380.119718,342.50953,1.245355
NH3,18.601273,2072.195311,334.731844,1.254117
CH4,50.025488,2225.00645,303.127074,1.250055
CH3OH,21.104194,2220.694862,304.292945,1.241727


In [23]:
# Compressible Flow Relations

# Convenient Function of the Ratio of Specific Heats
ga1 = lambda ga: ga / (ga - 1)

# Ratio of the total to the static temperature
theta = lambda M, ga: 1 + (ga-1)/2*M**2

# Ratio of the total to the static pressure
delta = lambda M, ga: theta(M, ga)**ga1(ga)

In [27]:
# Flight Mach number and altitude
Mach = 0.85; altitude_m = 10000

# Ambient Static Pressure & Temperature
Tamb_K = Atmosphere(altitude_m).temperature[0]
Pamb_bar = Atmosphere(altitude_m).pressure[0]/1e5
print(f"Ambient static temperature: {Tamb_K} K")
print(f"Ambient static pressure: {Pamb_bar} bar")

# Inlet total conditions
Tt_inlet = Tamb_K*theta(Mach, 1.4)
Pt_inlet = Pamb_bar*delta(Mach, 1.4)
print(f"\nInlet total temperature: {Tt_inlet} K")
print(f"Inlet total pressure: {Pt_inlet} bar")

# Compressor Pressure Ratio and Isentropic Efficiency
pi_c = 25; eta_c = 0.9
print(f"\nCompressor pressure ratio: {pi_c}")
print(f"Compressor isentropic efficiency: {eta_c}")

# Turbine Isentropic Efficiency
eta_t = 0.9
print(f"\nTurbine isentropic efficiency: {eta_t}")

Ambient static temperature: 223.25209264797857 K
Ambient static pressure: 0.2649987312280235 bar

Inlet total temperature: 255.51202003561144 K
Inlet total pressure: 0.4250099369001754 bar

Compressor pressure ratio: 25
Compressor isentropic efficiency: 0.9

Turbine isentropic efficiency: 0.9
